In [2]:
%load_ext sql

In [3]:
%sql sqlite://

Connecting to 'sqlite://'

In [4]:
%%sql
create table replayment (
    c_id int,
    due_date date,
    pay_date date);





Running query in 'sqlite://'

++
||
++
++

In [5]:
%%sql

insert into replayment(c_id,due_date,pay_date) values 
(1,'2025-01-01','2025-01-01'),(2,'2025-01-01','2025-01-06'),
(3,'2025-01-01','2025-01-01'),(4,'2025-01-01','2025-01-05'),
(5,'2025-01-01','2025-01-12'),(6,'2025-01-01','2025-01-03'),
(7,'2025-01-01','2025-02-01'),(8,'2025-01-01','2025-01-012'),
(9,'2025-01-01','2025-01-11'),(10,'2025-01-01','2025-02-01'),
(11,'2025-01-01','2025-01-18'),(12,'2025-01-01','2025-02-01');

Running query in 'sqlite://'

12 rows affected.

++
||
++
++

## Step 1 find date interval and group into range

In [6]:
%%sql

with group1 as (
    select c_id,due_date,pay_date,
    case
        when julianday(pay_date)-julianday(due_date) = 0 then 'Normal'
        when julianday(pay_date)-julianday(due_date) >=1 and julianday(pay_date)-julianday(due_date) <3 then 'S_overdue'
        when  julianday(pay_date)-julianday(due_date) >=3 and julianday(pay_date)-julianday(due_date) <10 then 'M_overdue'
        when  julianday(pay_date)-julianday(due_date) >=10 and julianday(pay_date)-julianday(due_date) <30 then 'L_overdue'
        else  'LL_overdue'
        END AS Range 
    from replayment)

select * from group1;
        

Running query in 'sqlite://'

c_id,due_date,pay_date,Range
1,2025-01-01,2025-01-01,Normal
2,2025-01-01,2025-01-06,M_overdue
3,2025-01-01,2025-01-01,Normal
4,2025-01-01,2025-01-05,M_overdue
5,2025-01-01,2025-01-12,L_overdue
6,2025-01-01,2025-01-03,S_overdue
7,2025-01-01,2025-02-01,LL_overdue
8,2025-01-01,2025-01-012,LL_overdue
9,2025-01-01,2025-01-11,L_overdue
10,2025-01-01,2025-02-01,LL_overdue


## Step 2 Count number of each group

In [7]:
%%sql
--Step 1
with group1 as (
    select c_id,due_date,pay_date,
    case
        when julianday(pay_date)-julianday(due_date) = 0 then 'Normal'
        when julianday(pay_date)-julianday(due_date) >=1 and julianday(pay_date)-julianday(due_date) <3 then 'S_overdue'
        when  julianday(pay_date)-julianday(due_date) >=3 and julianday(pay_date)-julianday(due_date) <10 then 'M_overdue'
        when  julianday(pay_date)-julianday(due_date) >=10 and julianday(pay_date)-julianday(due_date) <30 then 'L_overdue'
        else  'LL_overdue'
        END AS Range 
    from replayment),
--Step 2
group2 as (
    select count(*) as count1,Range from group1 group by Range order by count1)

select * from group2;

Running query in 'sqlite://'

count1,Range
1,S_overdue
2,M_overdue
2,Normal
3,L_overdue
4,LL_overdue


## Step3 Each group number divide by total number 

In [11]:
%%sql
with group1 as (
    select c_id,due_date,pay_date,
    case
        when julianday(pay_date)-julianday(due_date) = 0 then 'Normal'
        when julianday(pay_date)-julianday(due_date) >=1 and julianday(pay_date)-julianday(due_date) <3 then 'S_overdue'
        when  julianday(pay_date)-julianday(due_date) >=3 and julianday(pay_date)-julianday(due_date) <10 then 'M_overdue'
        when  julianday(pay_date)-julianday(due_date) >=10 and julianday(pay_date)-julianday(due_date) <30 then 'L_overdue'
        else  'LL_overdue'
        END AS Range 
    from replayment),
--Step 2
group2 as (
    select count(*) as count1,Range from group1 group by Range order by count1),
group3 as (
    select count(distinct c_id ) as unique_count from replayment ),
group4 as (
    select t1.range ,t1.count1, round(1.0*t1.count1 / t2.unique_count,4) as percentage 
    from group2 as t1  cross join group3 as t2)

select * from group4 order by percentage ;

Running query in 'sqlite://'

range,count1,percentage
S_overdue,1,0.0833
M_overdue,2,0.1667
Normal,2,0.1667
L_overdue,3,0.25
LL_overdue,4,0.3333
